In [13]:
import json
import sys
from openai import OpenAI
import pandas as pd
from anthropic import Anthropic


In [ ]:
#Put the inference outcome of the models you want to compare here, they will be combined into one big jsonl file. You can put as many as 7 model outcomes for LLMs to evaluate.
file_names = ['modelA_output.jsonl', 'modelB_output.jsonl','modelC_output']

# The new JSONL file to write the combined responses
output_file = 'combined_outcome.jsonl'

# Open the output file in write mode
with open(output_file, 'w', encoding='utf-8') as output_f:
    file_handles = [open(file_name, 'r', encoding='utf-8') for file_name in file_names]
    for lines in zip(*file_handles):
        responses = []
        for line in lines:
            entry = json.loads(line)
            response = entry.get('response')
            prompt = entry.get('prompt')
            if response:
                responses.append('[SEP]'+response)

        output_data = {"prompt": prompt, "response": " ".join(responses)}

        output_f.write(json.dumps(output_data)+ "\n")


In [3]:
claude_api = ''
def claude_accuracy_response(prompt):
    client = Anthropic(api_key=claude_api)
   
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=20,
        temperature=0,
        system="""You are a sports expert assigned to grade language models' generation performance on general sports-related text according to the provided rubric. 
        One prompt and five responses will be presented, all attempting to complete the same given prompt. Each response is seperated by [SEP] and limited to 80 tokens.

        Evaluate responses using the following rubric for "Accuracy and Factuality":
                    "1": "Mostly inaccurate, significant factual errors.",
                    "2": "Partially accurate, mix of correct and incorrect information.",
                    "3": "Mostly accurate, minor factual errors.",
                    "4": "Highly accurate, negligible errors.",
                    "5": "Fully accurate and factually impeccable."

        When evaluating, only consider how well the generated text continues or extends the given prompt in terms of context, topic, and style.
        Score these generated responses on a scale of 1-5. Only output the scores! Output scores in the following format: 'X, X, X, X, X', where X is a number between 1 and 5.""",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    
    response_str = response.content[0].text.strip()
    return response_str

In [4]:

def claude_relevance_response(prompt):
    client = Anthropic(api_key=claude_api)

    
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=20,
        temperature=0,
        system="""You are a sports expert assigned to grade language models' generation performance on general sports-related text according to the provided rubric. 
        One prompt and five responses will be presented, all attempting to complete the same given prompt. Each response is seperated by [SEP] and limited to 80 tokens.

        Evaluate responses using the following rubric for "Continuity and Relevance":
            "1": "Poor continuation, diverges significantly from the prompt's context or topic.",
            "2": "Weak continuation, maintains some elements of the prompt but introduces unrelated content.",
            "3": "Adequate continuation, generally follows the prompt's direction with some minor deviations.",
            "4": "Strong continuation, closely follows the prompt's context and style with minimal inconsistencies.",
            "5": "Excellent continuation, seamlessly extends the prompt's narrative, context, and style."

        When evaluating, only consider how well the generated text continues or extends the given prompt in terms of context, topic, and style.
        Score these generated responses on a scale of 1-5. Only output the scores! Output scores in the following format: 'X, X, X, X, X', where X is a number between 1 and 5.""",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    
    response_str = response.content[0].text.strip()
    return response_str

In [38]:

def OpenAI_relevence_response(client,prompt,num):
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages = [
            {
                "role": "system", 
                "content": f"""
                You are a sports expert assigned to grade language models' generation performance on general sports-related text according to the provided rubric. 
                1 prompt and {num} responses will be presented, all attempting to complete the same given prompt. Each response is seperated by [SEP] and limited to 80 tokens.

                Evaluate responses using the following rubric for "Continuity and Relevance":
                    "1": "Poor continuation, diverges significantly from the prompt's context or topic.",
                    "2": "Weak continuation, maintains some elements of the prompt but introduces unrelated content.",
                    "3": "Adequate continuation, generally follows the prompt's direction with some minor deviations.",
                    "4": "Strong continuation, closely follows the prompt's context and style with minimal inconsistencies.",
                    "5": "Excellent continuation, seamlessly extends the prompt's narrative, context, and style."

                When evaluating, only consider how well the generated text continues or extends the given prompt in terms of context, topic, and style.
                Score these generated responses on a scale of 1-5. Only output the scores! Output scores in the following format: 'X, X, X, X, X, X...', where X is a number between 1 and 5."""
            },
            {
                "role": "user", 
                "content": prompt
            }
        ],
        temperature = 0
    )
    ResponseStr = response.choices[0].message.content.strip()
    return ResponseStr

In [39]:

def OpenAI_accuracy_response(client,prompt,num):
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages = [
            {
                "role": "system", 
                "content": f"""
                You are a sports expert assigned to grade language models' generation performance on general sports-related text according to the provided rubric. 
                1 prompt and {num} responses will be presented, all attempting to complete the same given prompt. Each response is seperated by [SEP] and limited to 80 tokens.

                Evaluate responses using the following rubric for "Accuracy and Factuality":
                    "1": "Mostly inaccurate, significant factual errors.",
                    "2": "Partially accurate, mix of correct and incorrect information.",
                    "3": "Mostly accurate, minor factual errors.",
                    "4": "Highly accurate, negligible errors.",
                    "5": "Fully accurate and factually impeccable."

                When evaluating, only consider the accuracy and factuality in the context of the given prompt.
                Score these generated responses on a scale of 1-5. Only output the scores! Output scores in the following format: 'X, X, X, X, X, X...', where X is a number between 1 and 5."""
            },
            {
                "role": "user", 
                "content": prompt
            }
        ],
        temperature = 0
    )
    ResponseStr = response.choices[0].message.content.strip()
    return ResponseStr

In [42]:
client = OpenAI(api_key='')
counter =0
scores_acc = []
scores_rel = []
counter=0
with open('combined_outcome.jsonl', 'r', encoding='utf-8') as f:
    response_num = 8
    for line in f:
        entry = json.loads(line)
        counter+=1
        res = (f"""prompt: {entry['prompt']} \nResponse: {entry['response']}""")
        score_acc = OpenAI_accuracy_response(client,res,response_num).split(",")
        score_rel = OpenAI_relevence_response(client,res,response_num).split(",")
        if len(score_acc)==response_num:
            scores_acc.append(score_acc)
        if len(score_rel)==response_num:
            scores_rel.append(score_rel)
